<a href="https://colab.research.google.com/github/rar8393/NLG_tests/blob/main/roberta_model_finetune_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#https://github.com/huggingface/notebooks/blob/main/examples/language_modeling.ipynb

In [1]:
!pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 432 kB 16.4 MB/s 
     |████████████████████████████████| 4.9 MB 61.8 MB/s 
     |████████████████████████████████| 115 kB 60.6 MB/s 
     |████████████████████████████████| 163 kB 72.6 MB/s 
     |████████████████████████████████| 212 kB 67.5 MB/s 
     |████████████████████████████████| 127 kB 11.1 MB/s 
     |████████████████████████████████| 6.6 MB 57.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
import transformers

print(transformers.__version__)

4.22.2


In [5]:
from datasets import load_dataset
datasets = load_dataset("text", data_files={"train": 'train.txt', "validation": 'val.txt'})

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-d802eae2634af099/0.0.0/21a506d1b2b34316b1e82d0bd79066905d846e5d7e619823c0dd338d6f1fa6ad. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [7]:
show_random_elements(datasets["train"])

In [8]:
#"facebook/opt-125m". "saibo/legal-roberta-base"
model_checkpoint = "roberta-base"

In [9]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [10]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [11]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2429 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (936 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (769 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1698 > 512). Running this sequence through the model will result in indexing errors


#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1061 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (669 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1317 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (694 > 512). Running this sequence through the model will result in indexing errors


In [12]:
# block_size = tokenizer.model_max_length
block_size = 512

In [13]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [14]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [16]:
from transformers import AutoModelForCausalLM
#If you want to use `RobertaLMHeadModel` as a standalone, add `is_decoder=True.`
model = AutoModelForCausalLM.from_pretrained(model_checkpoint, is_decoder=True)

In [17]:
from transformers import Trainer, TrainingArguments

In [21]:
#https://stackoverflow.com/questions/72680734/huggingface-trainer-only-doing-3-epochs-no-matter-the-trainingarguments
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-health-test-1",
    evaluation_strategy = "steps", #epoch
    num_train_epochs=25, 
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=False,
    warmup_steps=1000,                # number of warmup steps for learning rate  
    save_steps=2000,
    eval_steps=500,
    save_total_limit=10,
    load_best_model_at_end= True,
    #output_dir='./results',
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [22]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

In [23]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3842
  Num Epochs = 25
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 12025


Step,Training Loss,Validation Loss
500,3.125700,3.122412
1000,3.063600,3.096858
1500,2.973500,3.061445
2000,2.902000,3.026058
2500,2.873600,3.006237
3000,2.799700,2.991123
3500,2.741300,2.989153
4000,2.695200,2.972284
4500,2.652300,2.968648
5000,2.608500,2.963950


***** Running Evaluation *****
  Num examples = 1097
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1097
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1097
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1097
  Batch size = 8
Saving model checkpoint to roberta-base-health-test-1/checkpoint-2000
Configuration saved in roberta-base-health-test-1/checkpoint-2000/config.json
Model weights saved in roberta-base-health-test-1/checkpoint-2000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1097
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1097
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1097
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1097
  Batch size = 8
Saving model checkpoint to roberta-base-health-test-1/checkpoint-4000
Configuration saved in roberta-base-health-test-1/checkpoint-4000/config.json
Model weights saved in roberta-base-health-test-1/checkpoint

TrainOutput(global_step=12025, training_loss=2.592085778757837, metrics={'train_runtime': 4683.1508, 'train_samples_per_second': 20.51, 'train_steps_per_second': 2.568, 'total_flos': 2.52866477495808e+16, 'train_loss': 2.592085778757837, 'epoch': 25.0})

In [24]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("roberta-base-health-test-1/checkpoint-12000")

loading configuration file roberta-base-health-test-1/checkpoint-12000/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base-health-test-1/checkpoint-12000",
  "architectures": [
    "RobertaForCausalLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": true,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.22.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file roberta-base-health-test-1/checkpoint-12000/pytorch_model.bin
All model checkpoint weights were used when initializing RobertaForCausalLM.

All

In [48]:
import importlib
import generate_txt
importlib.reload(generate_txt)
generator = generate_txt.generate_model_results(tokenizer, model)

In [29]:
test_metrics_1 = generator.generate_sample("The event has a ", visualize=False)

/content/generated_text_metrics.py:52: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity.append(gen_doc.similarity(sent))
/content/generated_text_metrics.py:54: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Span.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one 

In [30]:
for key in test_metrics_1.keys():
  print(key)

The event has a 
If you buy something through a link on this page, we may earn a small commission. This means that we may earn a small commission. Some alternative health products are available for purchase online. If you buy something through a link on this page, we may earn a small commission. How this works. People who use
The outbreak of SARS-CoV-2, caused by coronaviruses P and MERSERS, has impacted many countries and continents — especially more significantly, given that the outbreak has significantly affected the economic and social and psychological stability of the citizens. The outbreak started from Wuhan, China and remains
The coronavirus disease 2019 (COVID-19) is generating significant economic losses, and healthcare cost, including the provision of ICU beds and intensive care units (ICU). In March 2020, a small outbreak of Susceptible-Infectious-recovered-recovered (SEIR) policy
To identify and treat these early symptoms, a physician must perform a comprehensive, high-qua

In [31]:
for key in test_metrics_1.keys():
  print(test_metrics_1[key]['metric'])

{'number_ents': 0, 'ents': [], 'number_nouns': 1, 'nouns': ['event'], 'number_sents': 1, 'avg_sent_len': 4.0, 'sent_repeat_score': [0.0]}
{'number_ents': 0, 'ents': [], 'number_nouns': 12, 'nouns': ['link', 'page', 'commission', 'commission', 'health', 'product', 'purchase', 'online', 'link', 'page', 'commission', 'People'], 'number_sents': 6, 'avg_sent_len': 10.5, 'sent_repeat_score': [0.05555555555555558, 0.0, 0.0, 0.05555555555555558, 0.0, 0.0]}
{'number_ents': 3, 'ents': [MERSERS, Wuhan, China], 'number_nouns': 14, 'nouns': ['outbreak', 'SARS', 'CoV-2', 'coronaviruse', 'p', 'MERSERS', 'country', 'continent', 'outbreak', 'stability', 'citizen', 'outbreak', 'Wuhan', 'China'], 'number_sents': 2, 'avg_sent_len': 26.0, 'sent_repeat_score': [0.2790697674418605, 0.0]}
{'number_ents': 5, 'ents': [2019, ICU, ICU, March 2020, Susceptible-Infectious], 'number_nouns': 17, 'nouns': ['coronavirus', 'disease', 'COVID-19', 'loss', 'healthcare', 'cost', 'provision', 'ICU', 'bed', 'care', 'unit', 'I

In [32]:
print('Number named entities, number nouns, number of sentences, avg sentence length, repeat score')
for key in test_metrics_1.keys():
  print(test_metrics_1[key]['metric']['number_ents'], test_metrics_1[key]['metric']['number_nouns'],
        test_metrics_1[key]['metric']['number_sents'],  test_metrics_1[key]['metric']['avg_sent_len'],
        test_metrics_1[key]['metric']['sent_repeat_score'])

Number named entities, number nouns, number of sentences, avg sentence length, repeat score
0 1 1 4.0 [0.0]
0 12 6 10.5 [0.05555555555555558, 0.0, 0.0, 0.05555555555555558, 0.0, 0.0]
3 14 2 26.0 [0.2790697674418605, 0.0]
5 17 2 25.0 [0.19354838709677424, 0.1578947368421053]
2 15 2 30.5 [0.16000000000000003, 0.16666666666666663]
4 13 4 13.25 [0.06896551724137934, 0.0, 0.08333333333333337, 0.0]


In [33]:
test_metrics_2 = generator.generate_sample("The conference started with a ", visualize=False)

In [34]:
for key in test_metrics_2.keys():
  print(key)

The conference started with a 
Introduction: The SARS-CoV-2 virus is the novel virus responsible for the 2019-nCov disease, and it causes severe respiratory disease, and other respiratory disease.
The first COVID-19 patients presenting with mild to moderate symptoms, which were also hospitalized. Results were compared to those of those patients receiving general ventilation (GN), and in the control group for 15 days. These patients tended to have significant COVID-19 and were more pertained to a higher burden of disease
To provide an estimate of the basic reproduction number, we used daily new infections at the time of publication and the SARS-CoV-2 isolates. We estimated reproductive numbers by using raw numbers from confirmed COVID-19 cases and infected individuals from our community. We find that for both cases, the reproduction
Background The COVID-19 outbreak has affected the entire world. Methods Using RCTs from the WHO Health and Control Central Laboratories (WHO) from the WHO t

In [35]:
for key in test_metrics_2.keys():
  print(test_metrics_2[key]['metric'])

{'number_ents': 0, 'ents': [], 'number_nouns': 1, 'nouns': ['conference'], 'number_sents': 1, 'avg_sent_len': 5.0, 'sent_repeat_score': [0.0]}
{'number_ents': 1, 'ents': [2019-nCov], 'number_nouns': 9, 'nouns': ['introduction', 'SARS', 'CoV-2', 'virus', 'virus', 'nCov', 'disease', 'disease', 'disease'], 'number_sents': 1, 'avg_sent_len': 31.0, 'sent_repeat_score': [0.29032258064516125]}
{'number_ents': 2, 'ents': [first, 15 days], 'number_nouns': 14, 'nouns': ['covid-19', 'patient', 'symptom', 'result', 'patient', 'ventilation', 'GN', 'control', 'group', 'day', 'patient', 'covid-19', 'burden', 'disease'], 'number_sents': 3, 'avg_sent_len': 19.0, 'sent_repeat_score': [0.0, 0.04166666666666663, 0.05882352941176472]}
{'number_ents': 0, 'ents': [], 'number_nouns': 17, 'nouns': ['estimate', 'reproduction', 'number', 'infection', 'time', 'publication', 'SARS', 'CoV-2', 'isolate', 'number', 'number', 'covid-19', 'case', 'individual', 'community', 'case', 'reproduction'], 'number_sents': 3, 'a

In [36]:
print('Number named entities, number nouns, number of sentences, avg sentence length, repeat score')
for key in test_metrics_2.keys():
  print(test_metrics_2[key]['metric']['number_ents'], test_metrics_2[key]['metric']['number_nouns'],
        test_metrics_2[key]['metric']['number_sents'],  test_metrics_2[key]['metric']['avg_sent_len'],
        test_metrics_2[key]['metric']['sent_repeat_score'])

Number named entities, number nouns, number of sentences, avg sentence length, repeat score
0 1 1 5.0 [0.0]
1 9 1 31.0 [0.29032258064516125]
2 14 3 19.0 [0.0, 0.04166666666666663, 0.05882352941176472]
0 17 3 18.333333333333332 [0.11111111111111116, 0.10526315789473684, 0.0]
6 21 2 26.0 [0.09999999999999998, 0.26190476190476186]
0 16 4 15.25 [0.15384615384615385, 0.19999999999999996, 0.0714285714285714, 0.11111111111111116]


In [37]:
#event_txt = 'Calling All Climate Warriors! Youth for Climate Justice is hosting a series of webinars where we feature three climate justice-focused initiatives spearheaded by young community organizers. Discover local movements in your community, learn about effective ways to communicate with legislators and lead the charge in climate justice! '
sick_txt = 'The flu was really bad this year. Please prepare for the conference accordinly, vaccines are available. This will '


In [38]:
test_metrics_3 = generator.generate_sample(sick_txt, visualize=False)

In [39]:
for key in test_metrics_3.keys():
  print(key)

The flu was really bad this year. Please prepare for the conference accordinly, vaccines are available. This will 
The severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2) is causing a pandemic epidemic, caused by the novel coronavirus strain SARS-CoV-2. At the end of 2019, we described five subtypes of SARS-CoV-2. At present
Severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2) has become a global public health emergency, and there are wide- variety vaccination programs to protect against the virus. We develop a generalized deterministic linear regression model with the SARS-CoV-2 pandemic data from May
The spread of the SARS-CoV-2 virus has drastically escalated since late December 2019 and is a public health emergency worldwide, prompting more than 1.4 million confirmed infections (n=5,723). In this study, we analyzed the COVID-19 outbreak in India and found correlation between the
In this paper we explain the implementation and control of the current Spanish outbreak

In [40]:
for key in test_metrics_3.keys():
  print(test_metrics_3[key]['metric'])

{'number_ents': 1, 'ents': [this year], 'number_nouns': 4, 'nouns': ['flu', 'year', 'conference', 'vaccine'], 'number_sents': 3, 'avg_sent_len': 7.0, 'sent_repeat_score': [0.0, 0.0, 0.0]}
{'number_ents': 2, 'ents': [the end of 2019, five], 'number_nouns': 13, 'nouns': ['syndrome', 'coronavirus', 'SARS', 'CoV-2', 'epidemic', 'coronavirus', 'strain', 'SARS', 'CoV-2', 'end', 'subtype', 'SARS', 'CoV-2'], 'number_sents': 3, 'avg_sent_len': 15.0, 'sent_repeat_score': [0.2142857142857143, 0.06666666666666665, 0.0]}
{'number_ents': 1, 'ents': [May], 'number_nouns': 16, 'nouns': ['syndrome', 'coronavirus', 'SARS', 'CoV-2', 'health', 'emergency', 'variety', 'vaccination', 'program', 'virus', 'regression', 'model', 'SARS', 'CoV-2', 'datum', 'May'], 'number_sents': 2, 'avg_sent_len': 24.5, 'sent_repeat_score': [0.0, 0.0]}
{'number_ents': 4, 'ents': [late December 2019, more than 1.4 million, n=5,723, India], 'number_nouns': 14, 'nouns': ['spread', 'SARS', 'CoV-2', 'virus', 'December', 'health', 'e

In [41]:
print('Number named entities, number nouns, number of sentences, avg sentence length, repeat score')
for key in test_metrics_3.keys():
  print(test_metrics_3[key]['metric']['number_ents'], test_metrics_3[key]['metric']['number_nouns'],
        test_metrics_3[key]['metric']['number_sents'],  test_metrics_3[key]['metric']['avg_sent_len'],
        test_metrics_3[key]['metric']['sent_repeat_score'])

Number named entities, number nouns, number of sentences, avg sentence length, repeat score
1 4 3 7.0 [0.0, 0.0, 0.0]
2 13 3 15.0 [0.2142857142857143, 0.06666666666666665, 0.0]
1 16 2 24.5 [0.0, 0.0]
4 14 2 25.0 [0.02941176470588236, 0.125]
2 16 3 17.666666666666668 [0.10526315789473684, 0.12, 0.0]
5 14 3 17.0 [0.07692307692307687, 0.0, 0.0]


In [42]:
event_txt = 'Calling All Climate Warriors! Youth for Climate Justice is hosting a series of webinars where we feature three climate justice-focused initiatives spearheaded by young community organizers. Discover local movements in your community, learn about effective ways to communicate with legislators and lead the charge in climate justice! '
sick_txt = 'The flu was really bad this year. Please prepare for the conference accordinly, vaccines are available. This will '
#generator2(event_txt + sick_txt)
combo = event_txt + sick_txt

In [43]:
test_metrics_4 = generator.generate_sample(combo, visualize=False)

In [44]:
for key in test_metrics_4.keys():
  print(key)

Calling All Climate Warriors! Youth for Climate Justice is hosting a series of webinars where we feature three climate justice-focused initiatives spearheaded by young community organizers. Discover local movements in your community, learn about effective ways to communicate with legislators and lead the charge in climate justice! The flu was really bad this year. Please prepare for the conference accordinly, vaccines are available. This will 
Background: Despite rapid growth in the US. Covid-19 testing capacity, little is known about how effective effective isolation and isolation of aerosol driven transmission is and how these are affected by the impact of climate change. To evaluate the effectiveness of isolation and isolation for COVID-19, we conducted a series of
A new study finds that a number of medications that increase inflammation in the gut. In order to treat ulcerative colitis, a diet high in red blood cells helps break down the inflammation. However, it is difficult to tel

In [45]:
for key in test_metrics_4.keys():
  print(test_metrics_4[key]['metric'])

{'number_ents': 2, 'ents': [three, this year], 'number_nouns': 23, 'nouns': ['Climate', 'Warriors', 'youth', 'Climate', 'Justice', 'series', 'webinar', 'climate', 'justice', 'initiative', 'community', 'organizer', 'movement', 'community', 'way', 'legislator', 'charge', 'climate', 'justice', 'flu', 'year', 'conference', 'vaccine'], 'number_sents': 6, 'avg_sent_len': 12.333333333333334, 'sent_repeat_score': [0.0, 0.0, 0.04347826086956519, 0.0, 0.0, 0.0]}
{'number_ents': 1, 'ents': [US], 'number_nouns': 18, 'nouns': ['background', 'growth', 'US', 'Covid-19', 'testing', 'capacity', 'isolation', 'isolation', 'aerosol', 'transmission', 'impact', 'climate', 'change', 'effectiveness', 'isolation', 'isolation', 'covid-19', 'series'], 'number_sents': 3, 'avg_sent_len': 18.666666666666668, 'sent_repeat_score': [0.0, 0.22580645161290325, 0.125]}
{'number_ents': 0, 'ents': [], 'number_nouns': 14, 'nouns': ['study', 'number', 'medication', 'inflammation', 'gut', 'order', 'colitis', 'diet', 'blood', 

In [46]:
print('Number named entities, number nouns, number of sentences, avg sentence length, repeat score')
for key in test_metrics_4.keys():
  metric_load = test_metrics_4[key]['metric']
  print(metric_load['number_ents'], metric_load['number_nouns'],
        metric_load['number_sents'],  metric_load['avg_sent_len'],
        metric_load['sent_repeat_score'])

Number named entities, number nouns, number of sentences, avg sentence length, repeat score
2 23 6 12.333333333333334 [0.0, 0.0, 0.04347826086956519, 0.0, 0.0, 0.0]
1 18 3 18.666666666666668 [0.0, 0.22580645161290325, 0.125]
0 14 4 14.75 [0.125, 0.050000000000000044, 0.0625, 0.0]
1 15 2 26.5 [0.1428571428571429, 0.05555555555555558]
3 18 4 13.25 [0.2272727272727273, 0.2142857142857143, 0.0, 0.0]
3 16 2 22.0 [0.03703703703703709, 0.0]


In [49]:
#check different lengths
length_metric_1 = generator.generate_sample(sick_txt, max_length=32, visualize=False)
length_metric_2 = generator.generate_sample(sick_txt, max_length=64, visualize=False)
length_metric_3 = generator.generate_sample(sick_txt, max_length=128, visualize=False)

In [50]:
for key in length_metric_1.keys():
  print(key)
  print(len(key))

The flu was really bad this year. Please prepare for the conference accordinly, vaccines are available. This will 
114
Rapid sequencing, or deep sequencing software is the gold standard for identification of virus RNA in healthcare settings. The sequencing methodology involves using a DNA sequence sequence,
189
Background: The COVID-19 pandemic has resulted in a tremendous shortage of basic medical resources (AHs) in hospitals nationwide. These lack of medical
151
The recent outbreak of the SARS-CoV-2 coronavirus is now widespread, and many countries have prepared for preparations. Since December 2019,
140
Severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2), which emerged in Wuhan, China, has been successfully used
119
Introduction: The COVID-19 outbreak declared by the World Health Organization has had a dramatic impact on health care resources in many countries and has placed tremendous burdens
180


In [51]:
for key in length_metric_1.keys():
  print(length_metric_1[key]['metric'])

{'number_ents': 1, 'ents': [this year], 'number_nouns': 4, 'nouns': ['flu', 'year', 'conference', 'vaccine'], 'number_sents': 3, 'avg_sent_len': 7.0, 'sent_repeat_score': [0.0, 0.0, 0.0]}
{'number_ents': 0, 'ents': [], 'number_nouns': 14, 'nouns': ['sequencing', 'sequencing', 'software', 'gold', 'standard', 'identification', 'virus', 'RNA', 'healthcare', 'setting', 'methodology', 'dna', 'sequence', 'sequence'], 'number_sents': 2, 'avg_sent_len': 15.0, 'sent_repeat_score': [0.050000000000000044, 0.19999999999999996]}
{'number_ents': 0, 'ents': [], 'number_nouns': 8, 'nouns': ['background', 'covid-19', 'pandemic', 'shortage', 'resource', 'ah', 'hospital', 'lack'], 'number_sents': 2, 'avg_sent_len': 13.0, 'sent_repeat_score': [0.045454545454545414, 0.0]}
{'number_ents': 1, 'ents': [December 2019], 'number_nouns': 7, 'nouns': ['outbreak', 'SARS', 'CoV-2', 'coronavirus', 'country', 'preparation', 'December'], 'number_sents': 2, 'avg_sent_len': 12.5, 'sent_repeat_score': [0.04761904761904767

In [52]:
print('Number named entities, number nouns, number of sentences, avg sentence length, repeat score')
for key in length_metric_1.keys():
  metric_load = length_metric_1[key]['metric']
  print(metric_load['number_ents'], metric_load['number_nouns'],
        metric_load['number_sents'],  metric_load['avg_sent_len'],
        metric_load['sent_repeat_score'])

Number named entities, number nouns, number of sentences, avg sentence length, repeat score
1 4 3 7.0 [0.0, 0.0, 0.0]
0 14 2 15.0 [0.050000000000000044, 0.19999999999999996]
0 8 2 13.0 [0.045454545454545414, 0.0]
1 7 2 12.5 [0.04761904761904767, 0.0]
2 5 1 23.0 [0.08695652173913049]
1 12 1 28.0 [0.1071428571428571]


In [53]:
for key in length_metric_2.keys():
  print(key)

The flu was really bad this year. Please prepare for the conference accordinly, vaccines are available. This will 
If you buy something through a link on this page, we may earn a small commission. We may earn a small commission. How this works. A blood clot is where a blood clot clots and grows. The blood clot then travels across it, making it more likely to break, causing severe and life threatening complications
Severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2) has been recently recognized as a global health concern worldwide. Here, we examine the phylogenetic analysis of SARS-CoV-2 genomes from different locations. We have extracted the phylogenetic content of all studied genomes in China
The outbreak of COVID-19 is now a global pandemic with rising evidence to warrant the development of new vaccines for controlling the virus in several countries. We studied the epidemiological transmission patterns of COVID-19 across the whole SAE region. As of March 15, 2020, the number

In [54]:
for key in length_metric_2.keys():
    print(key)
    print(len(key))

The flu was really bad this year. Please prepare for the conference accordinly, vaccines are available. This will 
114
If you buy something through a link on this page, we may earn a small commission. We may earn a small commission. How this works. A blood clot is where a blood clot clots and grows. The blood clot then travels across it, making it more likely to break, causing severe and life threatening complications
302
Severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2) has been recently recognized as a global health concern worldwide. Here, we examine the phylogenetic analysis of SARS-CoV-2 genomes from different locations. We have extracted the phylogenetic content of all studied genomes in China
293
The outbreak of COVID-19 is now a global pandemic with rising evidence to warrant the development of new vaccines for controlling the virus in several countries. We studied the epidemiological transmission patterns of COVID-19 across the whole SAE region. As of March 15, 2020

In [55]:
print('Number named entities, number nouns, number of sentences, avg sentence length, repeat score')
for key in length_metric_2.keys():
  metric_load = length_metric_2[key]['metric']
  print(metric_load['number_ents'], metric_load['number_nouns'],
        metric_load['number_sents'],  metric_load['avg_sent_len'],
        metric_load['sent_repeat_score'])

Number named entities, number nouns, number of sentences, avg sentence length, repeat score
1 4 3 7.0 [0.0, 0.0, 0.0]
0 14 5 12.4 [0.05555555555555558, 0.0, 0.0, 0.33333333333333337, 0.09523809523809523]
1 14 3 16.666666666666668 [0.0, 0.0, 0.0]
1 16 3 18.0 [0.11111111111111116, 0.0714285714285714, 0.15384615384615385]
0 14 2 26.5 [0.0, 0.22857142857142854]
2 19 2 24.0 [0.045454545454545414, 0.1923076923076923]


In [56]:
for key in length_metric_3.keys():
  print(key)
  print(len(key))

The flu was really bad this year. Please prepare for the conference accordinly, vaccines are available. This will 
114
There are concerns that some groups are planning to limit or restrict the amount of healthcare provided in the country, especially for populations with mental health problems. In this article, we look at the possible impact of the lockdown and compare them with those implemented in other countries. We also describe some of the challenges that countries considering loosening the lockdowns around the world are facing. Some European countries have managed to put lockdown measures strictly to limit the spread of any disease, though there was some debate about some effects this would have on populations in the United States. In more detail on the effects of loosening restrictions on the most vulnerable population, we look
726
Introduction: We explored the impact of population-level epidemiological variables on COVID-19 transmission in Spain during the outbreak period. Using

In [57]:
for key in length_metric_3.keys():
  print(length_metric_3[key]['metric'])

{'number_ents': 1, 'ents': [this year], 'number_nouns': 4, 'nouns': ['flu', 'year', 'conference', 'vaccine'], 'number_sents': 3, 'avg_sent_len': 7.0, 'sent_repeat_score': [0.0, 0.0, 0.0]}
{'number_ents': 2, 'ents': [European, the United States], 'number_nouns': 30, 'nouns': ['concern', 'group', 'amount', 'healthcare', 'country', 'population', 'health', 'problem', 'article', 'impact', 'lockdown', 'country', 'challenge', 'country', 'lockdown', 'world', 'country', 'lockdown', 'measure', 'spread', 'disease', 'debate', 'effect', 'population', 'United', 'States', 'detail', 'effect', 'restriction', 'population'], 'number_sents': 5, 'avg_sent_len': 24.8, 'sent_repeat_score': [0.06896551724137934, 0.08695652173913049, 0.10526315789473684, 0.13888888888888884, 0.11764705882352944]}
{'number_ents': 11, 'ents': [Spain, May 2020, the first four weeks, 6 May 2020, the 27th day, 3.8%, 20-24-34 days, the first week, daily, R0, 0.4], 'number_nouns': 34, 'nouns': ['introduction', 'impact', 'population',

In [58]:
print('Number named entities, number nouns, number of sentences, avg sentence length, repeat score')
for key in length_metric_3.keys():
  metric_load = length_metric_3[key]['metric']
  print(metric_load['number_ents'], metric_load['number_nouns'],
        metric_load['number_sents'],  metric_load['avg_sent_len'],
        metric_load['sent_repeat_score'])

Number named entities, number nouns, number of sentences, avg sentence length, repeat score
1 4 3 7.0 [0.0, 0.0, 0.0]
2 30 5 24.8 [0.06896551724137934, 0.08695652173913049, 0.10526315789473684, 0.13888888888888884, 0.11764705882352944]
11 34 5 21.4 [0.045454545454545414, 0.1470588235294118, 0.0, 0.25, 0.15384615384615385]
2 5 1 15.0 [0.06666666666666665]
3 31 8 15.625 [0.10526315789473684, 0.0, 0.0, 0.09090909090909094, 0.21052631578947367, 0.15384615384615385, 0.13043478260869568, 0.0]
5 28 5 22.8 [0.0, 0.16129032258064513, 0.23684210526315785, 0.0, 0.0]


In [ ]:
#https://huggingface.co/docs/transformers/v4.16.2/en/custom_datasets

In [25]:
!cp -r roberta-base-health-test-1/checkpoint-12000/ roberta-health-test-v1/

In [26]:
!zip -r roberta_health_v1.zip roberta-health-test-v1/

  adding: roberta-health-test-v1/ (stored 0%)
  adding: roberta-health-test-v1/checkpoint-12000/ (stored 0%)
  adding: roberta-health-test-v1/checkpoint-12000/rng_state.pth (deflated 27%)
  adding: roberta-health-test-v1/checkpoint-12000/pytorch_model.bin (deflated 7%)
  adding: roberta-health-test-v1/checkpoint-12000/scheduler.pt (deflated 49%)
  adding: roberta-health-test-v1/checkpoint-12000/trainer_state.json (deflated 82%)
  adding: roberta-health-test-v1/checkpoint-12000/config.json (deflated 50%)
  adding: roberta-health-test-v1/checkpoint-12000/training_args.bin (deflated 48%)
  adding: roberta-health-test-v1/checkpoint-12000/optimizer.pt (deflated 8%)


In [27]:
from google.colab import files
files.download("roberta_health_v1.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>